In [4]:
import logging
import time
from gensim.models import Word2Vec
from indicnlp.tokenize import indic_tokenize

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Replace 'your_large_kannada_data.txt' with the path to your large Kannada data file
input_file_path = 'kn.txt/kn.txt'
output_file_path = 'tokenized_data.txt'
# Specify the batch size
batch_size = 10000

# Read and tokenize Kannada text in batches
with open(input_file_path, 'r', encoding='utf-8') as file:
    kannada_data = file.read()

    # Split the data into batches
    batches = [kannada_data[i:i+batch_size] for i in range(0, len(kannada_data), batch_size)]

    # Get the total number of batches
    total_batches = len(batches)

    # Initialize an empty list to store all tokenized sentences
    tokenized_data = []

    for i, batch in enumerate(batches, start=1):
        logging.info(f"Processing batch {i}/{total_batches}...")

        # Tokenize the batch
        tokenized_batch = indic_tokenize.trivial_tokenize(batch, lang='kn')

        # Append the tokenized batch to the list
        tokenized_data.append(tokenized_batch)
    # Save the tokenized batch to the output file
    with open(output_file_path, 'a', encoding='utf-8') as output_file:
        output_file.write(' '.join(tokenized_batch) + '\n')
        
    logging.info("Tokenization completed.")

MemoryError: 

In [1]:


import logging
from indicnlp.tokenize import indic_tokenize
import time
from gensim.models import Word2Vec
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

# Replace 'kn.txt' with the path to your large Kannada data file
input_file_path = 'kn.txt/kn.txt'
output_file_path = 'tokenized_data.txt'
# Open the output file in append mode
with open(output_file_path, 'a', encoding='utf-8') as output_file:
    # Read and tokenize Kannada text in batches
    with open(input_file_path, 'r', encoding='utf-8') as file:
        for i, line in enumerate(file, start=1):
            logging.info(f"Processing line {i}...")

            # Tokenize the line
            tokenized_line = indic_tokenize.trivial_tokenize(line, lang='kn')

            # Write the tokenized line to the output file
            output_file.write(' '.join(tokenized_line) + '\n')

logging.info("Tokenization and writing to file completed.")

PermissionError: [Errno 13] Permission denied: 'kn.txt'

In [ ]:
# The key modification is setting sg=0 in the Word2Vec constructor,
# which indicates the use of the CBOW model. If sg=1, it would imply the Skip-gram model.


import logging
from gensim.models import Word2Vec

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

def read_data(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            yield line.split()

# Create Word2Vec model with streaming
word2vec_model = Word2Vec(vector_size=100, window=5, min_count=1, workers=4)
# word2vec_model = Word2Vec(vector_size=100, window=5, min_count=1, workers=4, sg=0)
# by default its sg = 1(skip-gram)

data_filename = 'tokenized_data.txt'

# Build vocabulary
logging.info("Building vocabulary...")
word2vec_model.build_vocab(read_data(data_filename), progress_per=10000)

# Train the model using the streaming approach
logging.info("Training Word2Vec model...")
word2vec_model.train(read_data(data_filename), total_examples=word2vec_model.corpus_count, epochs=3)

# Save the trained Word2Vec model
output_model_path = 'output_word2vec_model.bin'
logging.info("Saving the trained Word2Vec model...")
word2vec_model.save(output_model_path)

logging.info("Word2Vec model training and saving completed.")


In [ ]:
from gensim.models import FastText
# from Fastext import
# Build vocabulary
# Initialize FastText model
Fasttext_model = FastText(size=100, window=5, min_count=1, workers=4)

# Build vocabulary
logging.info("Building vocabulary...")
Fasttext_model.build_vocab(sentences=read_data(data_filename), progress_per=10000)

# Train the model using the streaming approach
logging.info("Training FastText model...")
Fasttext_model.train(sentences=read_data(data_filename), total_examples=Fasttext_model.corpus_count, epochs=3)

# Save the trained FastText model
output_model_path = 'output_fasttext_model.bin'
logging.info("Saving the trained FastText model...")
Fasttext_model.save(output_model_path)

logging.info("FastText model training and saving completed.")

In [ ]:
from transformers import BertModel, BertTokenizer
import torch
import logging
import numpy as np

# Replace 'kn.txt' with the path to your tokenized data file
data_filename = 'kn.txt'

# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Create a list to store BERT embeddings
bert_embeddings = []

# Function to tokenize and get BERT embeddings for a sentence
def get_bert_embeddings(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
    return embeddings

# Iterate through your text data and get BERT embeddings
with open(data_filename, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()  # Remove leading/trailing whitespaces
        embeddings = get_bert_embeddings(line)
        bert_embeddings.append(embeddings)

# 'bert_embeddings' now contains the BERT embeddings for the sentences in your file

# Optionally, you can save the embeddings for later use
output_embeddings_bert = 'bert_embeddings.npy'
np.save(output_embeddings_bert, bert_embeddings)

logging.info("BERT embeddings extraction completed.")


In [ ]:
from transformers import RobertaModel, RobertaTokenizer
import torch
import logging

# Replace 'kn.txt' with the path to your tokenized data file
data_filename = 'kn.txt/kn.txt'

# Load pre-trained RoBERTa model and tokenizer
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaModel.from_pretrained(model_name)

# Create a list to store RoBERTa embeddings
roberta_embeddings = []

# Function to tokenize and get RoBERTa embeddings for a sentence
def get_roberta_embeddings(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().detach().numpy()
    return embeddings

# Iterate through your text data and get RoBERTa embeddings
with open(data_filename, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()  # Remove leading/trailing whitespaces
        embeddings = get_roberta_embeddings(line)
        roberta_embeddings.append(embeddings)

# 'roberta_embeddings' now contains the RoBERTa embeddings for the sentences in your file

# Optionally, you can save the embeddings for later use
output_embeddings_roberta = 'roberta_embeddings.npy'
np.save(output_embeddings_roberta, roberta_embeddings)

logging.info("RoBERTa embeddings extraction completed.")


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import logging

# Replace 'kn.txt' with the path to your tokenized data file
data_filename = 'kn.txt'

# Load pre-trained ELMo model from TensorFlow Hub
elmo_model_url = "https://tfhub.dev/google/elmo/3"
elmo = hub.load(elmo_model_url)

# Create a list to store ELMo embeddings
elmo_embeddings = []

# Function to get ELMo embeddings for a sentence
def get_elmo_embeddings(sentence):
    embeddings = elmo([sentence], signature="default", as_dict=True)["elmo"]
    embeddings = tf.reduce_mean(embeddings, axis=1).numpy()
    return embeddings

# Iterate through your text data and get ELMo embeddings
with open(data_filename, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()  # Remove leading/trailing whitespaces
        embeddings = get_elmo_embeddings(line)
        elmo_embeddings.append(embeddings)

# 'elmo_embeddings' now contains the ELMo embeddings for the sentences in your file

# Optionally, you can save the embeddings for later use
output_embeddings_elmo = 'elmo_embeddings.npy'
np.save(output_embeddings_elmo, elmo_embeddings)

logging.info("ELMo embeddings extraction completed.")


In [ ]:
words = [
    "ಹೆಚ್ಚಿನ",  # More
    "ಕಡಿಮೆ",  # Less
    "ಸಮಯ",  # Time
    "ದಿನ",  # Day
    "ವರ್ಷ",  # Year
    "ತಿಂಗಳು",  # Month
    "ಹೊಸ",  # New
    "ಹಳೆ",  # Old
    "ಜನ",  # People
    "ಊರ",  # Village
    "ನಗರ",  # City
    "ಹೊಸದು",  # Latest
    "ಮೊದಲ",  # First
    "ಕೊನೆ",  # Last
    "ಕೆಲಸ",  # Work
    "ವಿದ್ಯಾರ್ಥಿ",  # Student
    "ಶಿಕ್ಷಕ",  # Teacher
    # "ವಿದ್ಯಾನಿಲಯ",  # University
    "ರಾಜ್ಯ",  # State
    "ಸಾರ್ವಜನಿಕ",  # Public
    "ಗುರುತು",  # Mark
    "ಪರೀಕ್ಷೆ",  # Exam
    # "ವೇಶ್ಯಾಲಯ",  # School
    "ಆರೋಗ್ಯ",  # Health
    "ಆಹಾರ",  # Food
    "ವಿಜ್ಞಾನ",  # Science
    "ಕಲೆ",  # Art
    "ಸಾಹಿತ್ಯ",  # Literature
    "ಕೃಷಿ",  # Agriculture
    "ವಾಣಿಜ್ಯ",  # Commerce
    "ವಿಜ್ಞಾನ",  # Science
    "ವಾಣಿಜ್ಯ",  # Commerce
]


In [ ]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def visualize_word2vec_tsne(word2vec_model, words, dimensions=2, plot_size=(8, 6), annotate=True):
    """
    Visualize t-SNE plots for Word2Vec word embeddings.

    Parameters:
    - word2vec_model: The Word2Vec model.
    - words: A list of words for which you want to visualize embeddings.
    - dimensions: The number of dimensions for t-SNE (default is 2).
    - plot_size: The size of the plot (default is (8, 6)).
    - annotate: Whether to annotate points with word labels (default is True).

    Returns:
    - None (displays the plot).
    """
    embeddings = np.array([word2vec_model.wv[word] for word in words])

    # Apply t-SNE to reduce dimensionality
    tsne_model = TSNE(n_components=dimensions, random_state=42)
    tsne_results = tsne_model.fit_transform(embeddings)

    # Create a scatter plot for t-SNE results
    plt.figure(figsize=plot_size)
    plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c='blue', edgecolors='k', alpha=0.7)
    
    if annotate:
        for i, word in enumerate(words):
            plt.annotate(word, xy=(tsne_results[i, 0], tsne_results[i, 1]), fontsize=10, ha='right')

    plt.title('t-SNE Plot of Word2Vec Word Embeddings')
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.show()

# Example usage:
visualize_word2vec_tsne(word2vec_model, words)


In [ ]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def visualize_fasttext_tsne(fasttext_model, words, dimensions=2, plot_size=(8, 6), annotate=True):
    """
    Visualize t-SNE plots for FastText word embeddings.

    Parameters:
    - fasttext_model: The FastText model.
    - words: A list of words for which you want to visualize embeddings.
    - dimensions: The number of dimensions for t-SNE (default is 2).
    - plot_size: The size of the plot (default is (8, 6)).
    - annotate: Whether to annotate points with word labels (default is True).

    Returns:
    - None (displays the plot).
    """
    embeddings = np.array([fasttext_model.wv.get_vector(word) for word in words])

    # Apply t-SNE to reduce dimensionality
    tsne_model = TSNE(n_components=dimensions, random_state=42)
    tsne_results = tsne_model.fit_transform(embeddings)

    # Create a scatter plot for t-SNE results
    plt.figure(figsize=plot_size)
    plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c='green', edgecolors='k', alpha=0.7)
    
    if annotate:
        for i, word in enumerate(words):
            plt.annotate(word, xy=(tsne_results[i, 0], tsne_results[i, 1]), fontsize=10, ha='right')

    plt.title('t-SNE Plot of FastText Word Embeddings')
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.show()

# Example usage:
visualize_fasttext_tsne(Fasttext_model, words)


In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def visualize_bert_tsne(bert_embeddings, sentences, dimensions=2, plot_size=(8, 6), annotate=True):
    """
    Visualize t-SNE plots for BERT sentence embeddings.

    Parameters:
    - bert_embeddings: The BERT embeddings for sentences.
    - sentences: A list of sentences corresponding to the embeddings.
    - dimensions: The number of dimensions for t-SNE (default is 2).
    - plot_size: The size of the plot (default is (8, 6)).
    - annotate: Whether to annotate points with sentence labels (default is True).

    Returns:
    - None (displays the plot).
    """
    # Apply t-SNE to reduce dimensionality
    tsne_model = TSNE(n_components=dimensions, random_state=42)
    tsne_results = tsne_model.fit_transform(bert_embeddings)

    # Create a scatter plot for t-SNE results
    plt.figure(figsize=plot_size)
    plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c='blue', edgecolors='k', alpha=0.7)
    
    if annotate:
        for i, sentence in enumerate(sentences):
            plt.annotate(sentence[:15], xy=(tsne_results[i, 0], tsne_results[i, 1]), fontsize=8, ha='right')

    plt.title('t-SNE Plot of BERT Sentence Embeddings')
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.show()

# Example usage:
# Make sure to replace 'bert_embeddings.npy' with the actual path to your BERT embeddings file
bert_embeddings = np.load('bert_embeddings.npy', allow_pickle=True)
sentences_to_visualize = ['sentence1', 'sentence2', 'sentence3']  # Replace with your specific sentences
visualize_bert_tsne(bert_embeddings, sentences_to_visualize)


In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def visualize_bert_tsne(bert_embeddings, sentences, dimensions=2, plot_size=(8, 6), annotate=True):
    """
    Visualize t-SNE plots for BERT sentence embeddings.

    Parameters:
    - bert_embeddings: The BERT embeddings for sentences.
    - sentences: A list of sentences corresponding to the embeddings.
    - dimensions: The number of dimensions for t-SNE (default is 2).
    - plot_size: The size of the plot (default is (8, 6)).
    - annotate: Whether to annotate points with sentence labels (default is True).

    Returns:
    - None (displays the plot).
    """
    # Apply t-SNE to reduce dimensionality
    tsne_model = TSNE(n_components=dimensions, random_state=42)
    tsne_results = tsne_model.fit_transform(bert_embeddings)

    # Create a scatter plot for t-SNE results
    plt.figure(figsize=plot_size)
    plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c='blue', edgecolors='k', alpha=0.7)
    
    if annotate:
        for i, sentence in enumerate(sentences):
            plt.annotate(sentence[:15], xy=(tsne_results[i, 0], tsne_results[i, 1]), fontsize=8, ha='right')

    plt.title('t-SNE Plot of BERT Sentence Embeddings')
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.show()

# Example usage:
# Make sure to replace 'bert_embeddings.npy' with the actual path to your BERT embeddings file
bert_embeddings = np.load('bert_embeddings.npy', allow_pickle=True)
sample_sentences = ['ರಾಜ್ಯದ ವಿವಿಧ ಭಾಗಗಳಿಂದ ಹೋರಾಟಗಾರರು  ಈ ಅಧಿವೇಶನಕ್ಕೆ ಬಂದಿದ್ದರು.',
                           'ರಾಷ್ಟ್ರೀಯ ತರಬೇತುದಾರರಾದ ಜೇಸಿಐ ಸೆನೆಟರ್ ಪ್ರಮೋದ್‌ ಕುಮಾರ್‌ರವರು ತರಬೇತಿ ನೀಡಿದರು.', 
                           'ಅವರ ಹಾಡುಗಳು ಅನೇಕ ಸಂಯೋಜಿತ ಮತ್ತು ಉತ್ತಮ ಸಾಹಿತ್ಯಗಳನ್ನು ಒಳಗೊಂಡಿವೆ.'
                           'ಕೆಲವೊಮ್ಮೆ ಕುಟುಂಬ ಸದಸ್ಯರ ವಿರುದ್ಧವೇ ಪೈಪೋಟಿ ನಡೆಸಬೇಕಾಗುತ್ತೆ.',
                           '"ನನ್ನ ಮಗನಸಾವಿಗೆ ಸಂಬಂಧಿಸಿದಂತೆ ನನಗೆ ಯಾರ ಮೇಲಾದರೂ ಅನುಮಾನವಿದೆಯೇ ಎಂದು ಯಾರೂ ನನ್ನನ್ನು ವಿಚಾರಿಸುತ್ತಿಲ್ಲ.',
                           'ಮುಖಬೆಲೆಯ ಖೋಟಾನೋಟು ಕೊಟ್ಟು ಅಂಗಡಿಯ ಮಾಲೀಕರನ್ನು ಯಾಮಾರಿಸುತ್ತಿದ್ದ ಎನ್ನಲಾಗಿದೆ.',
                           'ಅವರ ಸ್ಥಾನಕ್ಕೆ ಯಾರೂ ನೇಮಕಗೊಂಡಿಲ್ಲ.',
                           'ಹಿಂದೂ ಅಥವಾ ಮುಸ್ಲಿಮರಿಗೆ ಈ ಕಾಯ್ದೆಗಳಿಂದ ಬಾಧಕವಿಲ್ಲ?',
                           'ಇನ್ನು ಆ ಅಮಾಯಕ ಹೆಂಗಸಿಗೆ ದಿಕ್ಕಾರು?',
                           'ಆಶಾದಾಯಕವಾಗಿ, ಓದುಗನಿಗೆ ಅರ್ಥ ಏನು "ಧನಾತ್ಮಕ" ಅರ್ಥಮಾಡಿಕೊಳ್ಳುವ.',
                           'ಹೆಸರೇ ಹೇಳುವಂತೆ ಈ ಮಂದಿರವನ್ನು ನಿಜವಾದ ಸ್ವರ್ಣದಿಂದಲೇ ಅಲಂಕರಿಸಲಾಗಿದೆ.',
                           'ನಾನು ಬಂದಿದ್ದೇನೆ',
                           'ಅವನು ಹಳೆಯ ಪುಸ್ತಕಗಳನ್ನು ಓದುತ್ತಾನೆ',
                           'ಹೊಸ ಸಾಹಿತ್ಯ ಪ್ರಕಟಣೆ ಬಹಳ ಹೆಚ್ಚು ಇದೆ'] 
visualize_bert_tsne(bert_embeddings, sample_sentences)


In [ ]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def visualize_roberta_tsne(roberta_model, sentences, dimensions=2, plot_size=(8, 6), annotate=True):
    """
    Visualize t-SNE plots for RoBERTa sentence embeddings.

    Parameters:
    - roberta_model: The RoBERTa model.
    - sentences: A list of sentences for which you want to visualize embeddings.
    - dimensions: The number of dimensions for t-SNE (default is 2).
    - plot_size: The size of the plot (default is (8, 6)).
    - annotate: Whether to annotate points with sentence labels (default is True).

    Returns:
    - None (displays the plot).
    """
    embeddings = np.array([get_roberta_embeddings(sentence) for sentence in sentences])

    # Apply t-SNE to reduce dimensionality
    tsne_model = TSNE(n_components=dimensions, random_state=42)
    tsne_results = tsne_model.fit_transform(embeddings)

    # Create a scatter plot for t-SNE results
    plt.figure(figsize=plot_size)
    plt.scatter(tsne_results[:, 0], tsne_results[:, 1], c='blue', edgecolors='k', alpha=0.7)
    
    if annotate:
        for i, sentence in enumerate(sentences):
            plt.annotate(sentence, xy=(tsne_results[i, 0], tsne_results[i, 1]), fontsize=10, ha='right')

    plt.title('t-SNE Plot of RoBERTa Sentence Embeddings')
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.show()

# Example usage:
# sample_sentences = ["ನಾನು ಬಂದಿದ್ದೇನೆ", "ಅವನು ಹಳೆಯ ಪುಸ್ತಕಗಳನ್ನು ಓದುತ್ತಾನೆ", "ಹೊಸ ಸಾಹಿತ್ಯ ಪ್ರಕಟಣೆ ಬಹಳ ಹೆಚ್ಚು ಇದೆ"]
visualize_roberta_tsne(model, sample_sentences)
